# English Auction

*Executive Summary*

This English Auction with five bidders and three rounds of bidding demonstrates the use of merkleization for Marlowe contracts. Because bidders may bid in any order and may bid more than once, unless they are disqualified by an illegal bid or timeout, the contract involves a combinatorial explosion of possibilities. Without merkleization, the contract JSON file is 990MB in size and contains 940k `Case` statements, but after merkleization the JSON file is 9.8MB and it contains just 1150 merkleized `Case` statements. The contract was both simulated and run on the Cardano blockchain: that involved 18 transactions.

*Characteristic of this contract*

*   A seller auctions one unit of an asset.
*   Any number of bidders bid on the contract.
*   Bids may occur in any order.
*   There are a fixed number of bids (rounds of bidding) allowed.
*   A bid is rejected if it isn't higher than all previous bids.
*   A bid is rejected if it isn't immediately followed by a deposit of the Lovelace that was bid.
*   Funds are returned to unsuccessful bidders.
*   There is deadline for depositing the asset.
*   Each bidding round has a deadline.

*Sequences of bids in this example*

1.  Third bidder bids 5 Ada.
2.  Second bidder bids 15 Ada.
3.  Third bidder bids 25 Ada.
4.  First bidder bids 30 Ada.
5.  Second bidder bids 40 Ada.
6.  Third bidder bids 50 Ada.

The third bidder wins the asset, the seller receives 50 Ada, and the other bidders receive back their deposits. Two of the five bidders do not bid.

## Preliminaries

Make a note of the versions of the Marlowe repository and software.

In [ ]:
marlowe-cli --version

marlowe-cli 0.0.5.0


In [ ]:
cardano-cli --version

cardano-cli 1.34.1 - linux-x86_64 - ghc-8.10
git rev 0000000000000000000000000000000000000000


In [ ]:
git rev-parse HEAD

de3ca53f994f989ba7f5693e5e03bdfbcd52beee


### Select the Cardano network

In [1]:
export CARDANO_TESTNET_MAGIC=1567
export CARDANO_NODE_SOCKET_PATH=node.socket

### Set up keys, addresses, and tokens

This section can be omitted if only off-chain simulation is performed.

#### Mint the native asset token

The bidding will be for a native token that represents either a digital or physical asset.

First create keys.

In [2]:
MINTING_SKEY=english-auction-x.skey
cardano-cli address key-gen --signing-key-file $MINTING_SKEY --verification-key-file english-auction-x.vkey
MINTING_ADDRESS=$(cardano-cli address build --testnet-magic "$CARDANO_TESTNET_MAGIC" --payment-verification-key-file english-auction-x.vkey)
echo $MINTING_ADDRESS

addr_test1vqryjqrp77xt97naa7ez5qvtqvkuucyyzqw02vl9vxmu4vq7gkjkv


For terseness, denominate bids in Ada instead of Lovelace.

In [3]:
ADA=1000000

Fund the address.

In [4]:
marlowe-cli util faucet --lovelace $((50 * ADA)) \
                        --out-file /dev/null     \
                        --submit 600             \
                        $MINTING_ADDRESS

TxId "e0efe318c28d92b82ec97d5cdb562c8ae65cf3525ea7b5df5b4ef1faba557b6f"


Mint the token.

In [5]:
TOKEN_NAME=Globe

In [6]:
TOKEN_CURRENCY=$(
marlowe-cli util mint --required-signer $MINTING_SKEY   \
                      --change-address $MINTING_ADDRESS \
                      --out-file /dev/null              \
                      --submit 600                      \
                      $TOKEN_NAME                       \
| sed -e 's/^PolicyID "\(.*\)"$/\1/   '                 \
)

In [7]:
TOKEN=$TOKEN_CURRENCY.$TOKEN_NAME
echo "The subject of the auction is $TOKEN."
echo "The hexadecimal representation of this is $TOKEN_CURRENCY.$(echo -n $TOKEN_NAME | basenc --base16)."

The subject of the auction is 76574a57d6a0bc1db5ad534b90ac0aacdad765754eca3f4e7dc7c116.Globe.
The hexadecimal representation of this is 76574a57d6a0bc1db5ad534b90ac0aacdad765754eca3f4e7dc7c116.476C6F6265.


#### Generate keys for parties

The seller has role index 0, and the bidders have role indices matching their sequence.

In [8]:
ROLE=("Seller" "Bidder 1" "Bidder 2" "Bidder 3")

In [9]:
for i in 0 1 2 3
do
  SKEY[$i]=english-auction-$i.skey
  cardano-cli address key-gen --signing-key-file ${SKEY[$i]} --verification-key-file english-auction-$i.vkey
  ADDRESS[$i]=$(cardano-cli address build --testnet-magic "$CARDANO_TESTNET_MAGIC" --payment-verification-key-file english-auction-$i.vkey)
  echo "${ROLE[$i]} has address ${ADDRESS[$i]}."
done

Seller has address addr_test1vr8j4ev357a2q6kjhwqg6ejclwv6mjrewrtn3szm08fjxfc9n009p.
Bidder 1 has address addr_test1vrkxylr7k0zfdr7xf8h42lmafh7mlzlrcz4smf2e2he6x9q4vsa9t.
Bidder 2 has address addr_test1vrls090up80k509v82x8h8puqrduargxr5cmtydqyttqjvg5u4g3t.
Bidder 3 has address addr_test1vrvstcvdp9fugqswl8ftljv03zmjpdulvwvv074neaewp2ss9gwvg.


#### Fund the addresses

In [10]:
marlowe-cli util faucet --lovelace $((100 * ADA)) --out-file /dev/null --submit 600  ${ADDRESS[@]}

TxId "163e7181763ed971204ea69fb794696da1e9bd10be50eb069819efd61bcce818"


#### Mint the role tokens

In [11]:
ROLES_CURRENCY=$(
marlowe-cli util mint --required-signer ${SKEY[0]}   \
                      --change-address ${ADDRESS[0]} \
                      --out-file /dev/null           \
                      --submit 600                   \
                      "${ROLE[@]}"                   \
| sed -e 's/^PolicyID "\(.*\)"$/\1/'                 \
)
echo The roles currency has policy ID $ROLES_CURRENCY.

The roles currency has policy ID d6390543e76fe398ccb5d7e90da5f0548e29759cf94d565d82ad4b19.


In [12]:
for i in 0 1 2 3
do
  ROLE_TOKEN[$i]="$ROLES_CURRENCY.${ROLE[$i]}"
done

View the role tokens.

In [13]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address ${ADDRESS[0]}

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
3aced0ba2210ae7a4e644d9285c986f08fea126b3cf353305bd85ee0db6e5440     0        59815667 lovelace + TxOutDatumNone
3aced0ba2210ae7a4e644d9285c986f08fea126b3cf353305bd85ee0db6e5440     1        10000000 lovelace + 1 d6390543e76fe398ccb5d7e90da5f0548e29759cf94d565d82ad4b19.4269646465722031 + TxOutDatumNone
3aced0ba2210ae7a4e644d9285c986f08fea126b3cf353305bd85ee0db6e5440     2        10000000 lovelace + 1 d6390543e76fe398ccb5d7e90da5f0548e29759cf94d565d82ad4b19.4269646465722032 + TxOutDatumNone
3aced0ba2210ae7a4e644d9285c986f08fea126b3cf353305bd85ee0db6e5440     3        10000000 lovelace + 1 d6390543e76fe398ccb5d7e90da5f0548e29759cf94d565d82ad4b19.4269646465722033 + TxOutDatumNone
3aced0ba2210ae7a4e644d9285c986f08fea126b3cf353305bd85ee0db6e5440     4        10000000 lovelace + 1 d6390543e76fe398ccb5d7e90da5f0548e29759cf9

#### Distribute the role tokens

In [14]:
marlowe-cli transaction simple --required-signer ${SKEY[0]} \
                               --change-address ${ADDRESS[0]} \
                               --tx-in $(marlowe-cli util select --asset-only "${ROLE_TOKEN[1]}" ${ADDRESS[0]} | sed -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/') \
                               --tx-in $(marlowe-cli util select --asset-only "${ROLE_TOKEN[2]}" ${ADDRESS[0]} | sed -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/') \
                               --tx-in $(marlowe-cli util select --asset-only "${ROLE_TOKEN[3]}" ${ADDRESS[0]} | sed -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/') \
                               --tx-out "${ADDRESS[1]}+2000000+1 ${ROLE_TOKEN[1]}" \
                               --tx-out "${ADDRESS[2]}+2000000+1 ${ROLE_TOKEN[2]}" \
                               --tx-out "${ADDRESS[3]}+2000000+1 ${ROLE_TOKEN[3]}" \
                               --out-file /dev/null \
                               --submit 600

TxId "6db1c167b9f9d04dff91a1d5a5592c3cf210bf224d76b247f5726429d4ce7ca8"


#### Send the native token for the asset to the seller

In [15]:
marlowe-cli transaction simple --required-signer $MINTING_SKEY \
                               --change-address $MINTING_ADDRESS \
                               --tx-in $(marlowe-cli util select --asset-only $TOKEN $MINTING_ADDRESS | sed -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/') \
                               --tx-out "${ADDRESS[0]}+2000000+1 $TOKEN" \
                               --out-file /dev/null \
                               --submit 600

TxId "c271ad69a6231e950a16c10e20b10e2581015fb5e3f6dca91c2d7bbec29590f9"


#### Check that the roles have the tokens they need

In [16]:
for i in 0 1 2 3
do
  echo
  echo "${ROLE[$i]} possesses the following UTxOs:"
  cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address ${ADDRESS[$i]}
done


Seller possesses the following UTxOs:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
3aced0ba2210ae7a4e644d9285c986f08fea126b3cf353305bd85ee0db6e5440     0        59815667 lovelace + TxOutDatumNone
3aced0ba2210ae7a4e644d9285c986f08fea126b3cf353305bd85ee0db6e5440     4        10000000 lovelace + 1 d6390543e76fe398ccb5d7e90da5f0548e29759cf94d565d82ad4b19.53656c6c6572 + TxOutDatumNone
6db1c167b9f9d04dff91a1d5a5592c3cf210bf224d76b247f5726429d4ce7ca8     0        23811663 lovelace + TxOutDatumNone
c271ad69a6231e950a16c10e20b10e2581015fb5e3f6dca91c2d7bbec29590f9     1        2000000 lovelace + 1 76574a57d6a0bc1db5ad534b90ac0aacdad765754eca3f4e7dc7c116.476c6f6265 + TxOutDatumNone

Bidder 1 possesses the following UTxOs:
                           TxHash                                 TxIx        Amount
------------------------------------------------------------------

### Set up the contract

Start the contract at the current time.

In [17]:
NOW=$(($(date -u +%s) * 1000))
echo $NOW

1655691097000


Use a four hour time window for submitting transactions. This is just for convenienc when testing this contract.

In [18]:
HOUR=$((60 * 60 * 1000))
LATER=$((NOW + 4 * HOUR))
echo $LATER

1655705497000


There is a minimum Ada requirement associated with a native token.

In [19]:
MIN_ADA=$((2 * ADA))

Create a Haskell program that will generate a contract with six rounds of bidding for three bidders. We could use `runhaskell` instead of `ghc`, but `runhaskell` is slow because it doesn't perform optimizations.

In [20]:
cat << EOI > english-auction.hs

{-# LANGUAGE NumericUnderscores #-}
{-# LANGUAGE OverloadedStrings  #-}
{-# LANGUAGE Trustworthy        #-}

module Main (
-- * Entry point
  main
-- * Contracts
, makeContract
) where

import Data.Aeson (encodeFile)
import Data.String (fromString)
import Language.Marlowe.Extended.V1
import qualified Language.Marlowe.Core.V1.Semantics.Types as S (Contract)

-- | Print the contract.
main :: IO ()
main =
  encodeFile "english-auction-01.contract"
    . (toCore :: Contract -> Maybe S.Contract)
    $ makeContract 6 5
      (Bound 2_000_000 1_000_000_000_000)
      (Token "$TOKEN_CURRENCY" "$TOKEN_NAME")
      $NOW (5 * $HOUR)

-- | The party that sells the item at auction.
seller :: Party
seller = Role "Seller"

-- | The quantity of items that is auctioned.
assetAmount :: Value
assetAmount = Constant 1

-- | The minimum Ada requirement for the asset.
minAda :: Value
minAda = Constant $MIN_ADA

-- | The value of the highest bid.
highestBid :: ValueId
highestBid = "Highest Bid"

-- | Create the Marlowe contract for an English auction.
makeContract :: Int       -- ^ The number of rounds of bidding.
             -> Int       -- ^ The number of bidders.
             -> Bound     -- ^ The range for valid bids, in Lovelace.
             -> Token     -- ^ The token representing the asset being bid upon.
             -> Integer   -- ^ The start time of the contract.
             -> Integer   -- ^ The spacing between bid deadlines.
             -> Contract  -- ^ The English auction.
makeContract nRounds nBidders bidBounds assetToken start delta =
  let
    bids =
      [
        ChoiceId (fromString $ "Bid " <> show i) party
      |
        i <- [1..nBidders]
      , let party = fromString $ "Bidder " <> show i
      ]
    deadlines =
      [
        POSIXTime $ start + delta * (1 + fromIntegral i)
      |
        i <- [1..nRounds]
      ]
  in
   -- Deposit the asset, then make the bids, but close if no one bids.
    makeAssetDeposit assetToken (POSIXTime $ start + delta)
      $ makeBids bidBounds assetToken deadlines bids
        Close

-- | Deposit the asset that is the subject of the bidding.
makeAssetDeposit :: Token     -- ^ The token representing the asset being bid upon.
                 -> Timeout   -- ^ The timeout for depositing the asset.
                 -> Contract  -- ^ The contract to be executed after the asset is deposited.
                 -> Contract  -- ^ The contract for the asset deposit and subsequent activity.
makeAssetDeposit asset assetDeadline continuation =
  When
    [
      -- The seller deposits the asset being auctioned.
      Case (Deposit seller seller asset assetAmount)
        continuation
    ]
    -- The contract ends if the deposit is not made.
    assetDeadline
    Close

-- | Make the contract for bids.
makeBids :: Bound       -- ^ The range of valid bids, in Lovelace.
         -> Token       -- ^ The token representing the asset being bid upon.
         -> [Timeout]   -- ^ The deadlines for the rounds of bidding.
         -> [ChoiceId]  -- ^ The choices the bidders will make.
         -> Contract    -- ^ The contract to be executed at the end of the bidding.
         -> Contract    -- ^ The bidding contract.
makeBids _ _ [] _ continuation = continuation
makeBids _ _ _ [] continuation = continuation
makeBids bounds assetToken (deadline : remainingDeadlines) bids continuation =
  When
    [
      -- Let the bidder make their bid.
      Case (Choice bid [bounds])
        $ let
            bidAmount = ChoiceValue bid
          in
            -- Check if the bid is highest so far.
            If (ValueGT bidAmount $ UseValue highestBid)
              -- Require a deposit if the bid is highest.
              (
                When
                  [
                    -- Deposit the Lovelace for the bid.
                    Case (Deposit bidder bidder ada . SubValue (AddValue bidAmount minAda) $ AvailableMoney bidder ada)
                      -- Record the new highest amount.
                      $ Let highestBid bidAmount
                      -- Handle the remaining bids.
                      $ remaining
                      -- Make the payment for the asset.
                      $ Pay bidder (Party seller) ada bidAmount
                      $ Pay seller (Party bidder) assetToken assetAmount
                        Close
                  ]
                  -- Ignore the bid and disqualify the bidder if the deposit was not made.
                  deadline
                    $ disqualify continuation
              )
              -- Ignore the bid and disqualify the bidder if it is not highest.
              (
                -- Handle the remaining bids and finalization.
                disqualify continuation
              )
    |
      bid@(ChoiceId _ bidder) <- bids
    , let remaining = makeBids bounds assetToken remainingDeadlines bids
          disqualify = makeBids bounds assetToken remainingDeadlines $ filter (/= bid) bids

    ]
    -- End the bidding if no one bids in this round.
    deadline
    continuation
    
EOI

👉 Note that the above contract contains the multiple-input vulnerability where a bidder could submit all of the contract's bids in a single transaction, thus locking other bidders out of participating. How would you simply alter the contract to prevent this?

In [21]:
ghc -O2 -o english-auction english-auction.hs

[1 of 1] Compiling Main             ( english-auction.hs, english-auction.o )
Linking english-auction ...


In [22]:
`which time` \
./english-auction

57.81user 8.02system 1:15.89elapsed 86%CPU (0avgtext+0avgdata 13986688maxresident)k
2443904inputs+2018096outputs (75289major+4519845minor)pagefaults 0swaps


How large is the contract?

In [23]:
ls -lh english-auction-01.contract

-rw-rw-r-- 1 bbush bbush-upg 985M Jun 19 20:13 english-auction-01.contract


How many `Case` statements does it contain?

In [105]:
sed -e 's/case/&\n/g' english-auction-01.contract | grep case | wc -l

942181


### Set the initial state

The Seller will deposit 2 Ada when the contract is created.

In [24]:
yaml2json << EOI > english-auction-01.state
accounts:
- - - role_token: Seller
    - currency_symbol: ''
      token_name: ''
  - $MIN_ADA
boundValues: []
choices: []
minTime: $NOW
EOI

In [25]:
cat english-auction-01.state

{"accounts":[[[{"role_token":"Seller"},{"currency_symbol":"","token_name":""}],2000000]],"boundValues":[],"choices":[],"minTime":1655691097000}


## Simulate the Contract

We start by simulating the contract before running on the blockchain. This lets us test our inputs without committing any Ada. It also separates the logic of the contract's progression from the details of eUTxO management.

### 1. Initialize the contract information

Create a `.marlowe` file that contains all of the information needed to start the contract. Note that we merkleization the contract because of its large size.

In [26]:
`which time` \
marlowe-cli run initialize --roles-currency $ROLES_CURRENCY            \
                           --contract-file english-auction-01.contract \
                           --state-file    english-auction-01.state    \
                           --out-file      english-auction-01.marlowe  \
                           --merkleize                                 \
                           --print-stats


Validator size: 12773
Base-validator cost: ExBudget {exBudgetCPU = ExCPU 24741463, exBudgetMemory = ExMemory 83200}
303.31user 52.67system 10:44.46elapsed 55%CPU (0avgtext+0avgdata 13905708maxresident)k
34809824inputs+20168outputs (2041852major+9429868minor)pagefaults 0swaps


The contract now contains just the native asset, in the Seller's account.

In [27]:
jq .state english-auction-01.marlowe | json2yaml

accounts:
- - - role_token: Seller
    - currency_symbol: ''
      token_name: ''
  - 2000000
boundValues: []
choices: []
minTime: 1655691097000


How large is the merkleized contract?

In [106]:
ls -lh english-auction-01.marlowe

-rw-rw-r-- 1 bbush bbush-upg 9.8M Jun 19 20:23 english-auction-01.marlowe


How many merkle hashes does it contain?

In [107]:
jq '.continuations | length' english-auction-01.marlowe

1151


### 2. The Seller deposits the native asset

In [28]:
marlowe-cli run prepare --deposit-account Role=Seller             \
                        --deposit-party   Role=Seller             \
                        --deposit-token "$TOKEN"                  \
                        --deposit-amount 1                        \
                        --invalid-before $NOW                     \
                        --invalid-hereafter $LATER                \
                        --marlowe-file english-auction-01.marlowe \
                        --out-file     english-auction-02.marlowe \
                        --print-stats


Datum size: 545


Now the contract contains initial Ada and the native token, both in the Seller's account.

In [29]:
jq .state english-auction-02.marlowe | json2yaml

accounts:
- - - role_token: Seller
    - currency_symbol: ''
      token_name: ''
  - 2000000
- - - role_token: Seller
    - currency_symbol: 76574a57d6a0bc1db5ad534b90ac0aacdad765754eca3f4e7dc7c116
      token_name: Globe
  - 1
boundValues: []
choices: []
minTime: 1655691097000


### 3. Third bidder bids 5 Ada

In [30]:
AMOUNT_1=$((5 * ADA))

In [31]:
marlowe-cli run prepare --choice-party Role="Bidder 3"            \
                        --choice-name "Bid 3"                     \
                        --choice-number $AMOUNT_1                 \
                        --invalid-before $NOW                     \
                        --invalid-hereafter $LATER                \
                        --marlowe-file english-auction-02.marlowe \
                        --out-file     english-auction-03.marlowe \
                        --print-stats


Datum size: 645


The bid has been recorded in the state.

In [32]:
jq .state english-auction-03.marlowe | json2yaml

accounts:
- - - role_token: Seller
    - currency_symbol: ''
      token_name: ''
  - 2000000
- - - role_token: Seller
    - currency_symbol: 76574a57d6a0bc1db5ad534b90ac0aacdad765754eca3f4e7dc7c116
      token_name: Globe
  - 1
boundValues: []
choices:
- - choice_name: Bid 3
    choice_owner:
      role_token: Bidder 3
  - 5000000
minTime: 1655691097000


### 4. Third bidder deposits 7 Ada

The bidder must deposit whatever Ada they just bid, along with the minimum Ada requirement.

In [33]:
DELTA_1=$((AMOUNT_1 + $MIN_ADA))
echo $DELTA_1

7000000


In [34]:
marlowe-cli run prepare --deposit-account Role="Bidder 3"         \
                        --deposit-party Role="Bidder 3"           \
                        --deposit-amount $DELTA_1                 \
                        --invalid-before $NOW                     \
                        --invalid-hereafter $LATER                \
                        --marlowe-file english-auction-03.marlowe \
                        --out-file     english-auction-04.marlowe \
                        --print-stats


Datum size: 765


The third bidder's deposit is in their account.

In [35]:
jq .state english-auction-04.marlowe | json2yaml

accounts:
- - - role_token: Seller
    - currency_symbol: ''
      token_name: ''
  - 2000000
- - - role_token: Seller
    - currency_symbol: 76574a57d6a0bc1db5ad534b90ac0aacdad765754eca3f4e7dc7c116
      token_name: Globe
  - 1
- - - role_token: Bidder 3
    - currency_symbol: ''
      token_name: ''
  - 7000000
boundValues:
- - Highest Bid
  - 5000000
choices:
- - choice_name: Bid 3
    choice_owner:
      role_token: Bidder 3
  - 5000000
minTime: 1655691097000


### 5. Second bidder bids 15 Ada

In [36]:
AMOUNT_2=$((15 * ADA))

In [37]:
marlowe-cli run prepare --choice-party Role="Bidder 2"            \
                        --choice-name "Bid 2"                     \
                        --choice-number $AMOUNT_2                 \
                        --invalid-before $NOW                     \
                        --invalid-hereafter $LATER                \
                        --marlowe-file english-auction-04.marlowe \
                        --out-file     english-auction-05.marlowe \
                        --print-stats


Datum size: 865


The second bid has been recorded.

In [38]:
jq .state.choices english-auction-05.marlowe | json2yaml

- - choice_name: Bid 3
    choice_owner:
      role_token: Bidder 3
  - 5000000
- - choice_name: Bid 2
    choice_owner:
      role_token: Bidder 2
  - 15000000


### 6. Second bidder deposits 17 Ada

In [39]:
DELTA_2=$((AMOUNT_2 + MIN_ADA))
echo $DELTA_2

17000000


In [40]:
marlowe-cli run prepare --deposit-account Role="Bidder 2"         \
                        --deposit-party Role="Bidder 2"           \
                        --deposit-amount $DELTA_2                 \
                        --invalid-before $NOW                     \
                        --invalid-hereafter $LATER                \
                        --marlowe-file english-auction-05.marlowe \
                        --out-file     english-auction-06.marlowe \
                        --print-stats


Warnings:
  TransactionShadowing "Highest Bid" 5000000 15000000
Datum size: 821


The second bidder's deposit appears in their account.

In [41]:
jq .state.accounts english-auction-06.marlowe | json2yaml

- - - role_token: Seller
    - currency_symbol: ''
      token_name: ''
  - 2000000
- - - role_token: Seller
    - currency_symbol: 76574a57d6a0bc1db5ad534b90ac0aacdad765754eca3f4e7dc7c116
      token_name: Globe
  - 1
- - - role_token: Bidder 3
    - currency_symbol: ''
      token_name: ''
  - 7000000
- - - role_token: Bidder 2
    - currency_symbol: ''
      token_name: ''
  - 17000000


### 7. Third bidder bids 25 Ada

In [42]:
AMOUNT_3=$((25 * ADA))

In [43]:
marlowe-cli run prepare --choice-party Role="Bidder 3"            \
                        --choice-name "Bid 3"                     \
                        --choice-number $AMOUNT_3                 \
                        --invalid-before $NOW                     \
                        --invalid-hereafter $LATER                \
                        --marlowe-file english-auction-06.marlowe \
                        --out-file     english-auction-07.marlowe \
                        --print-stats


Datum size: 893


The record of choices reflects the third bidder's increased bid.

In [44]:
jq .state.choices english-auction-07.marlowe | json2yaml

- - choice_name: Bid 3
    choice_owner:
      role_token: Bidder 3
  - 25000000
- - choice_name: Bid 2
    choice_owner:
      role_token: Bidder 2
  - 15000000


### 8. Third bidder deposits 20 Ada

Because there is already 5 Ada in the third bidder's account, they only need to deposit 20 Ada more, not the full 25 Ada that they bid.

In [45]:
DELTA_3=$((AMOUNT_3 - AMOUNT_1))
echo $DELTA_3

20000000


In [46]:
marlowe-cli run prepare --deposit-account Role="Bidder 3"         \
                        --deposit-party Role="Bidder 3"           \
                        --deposit-amount $DELTA_3                 \
                        --invalid-before $NOW                     \
                        --invalid-hereafter $LATER                \
                        --marlowe-file english-auction-07.marlowe \
                        --out-file     english-auction-08.marlowe \
                        --print-stats


Warnings:
  TransactionShadowing "Highest Bid" 15000000 25000000
Datum size: 821


The third bidder's account has been updated.

In [47]:
jq .state.accounts english-auction-08.marlowe | json2yaml

- - - role_token: Seller
    - currency_symbol: ''
      token_name: ''
  - 2000000
- - - role_token: Seller
    - currency_symbol: 76574a57d6a0bc1db5ad534b90ac0aacdad765754eca3f4e7dc7c116
      token_name: Globe
  - 1
- - - role_token: Bidder 3
    - currency_symbol: ''
      token_name: ''
  - 27000000
- - - role_token: Bidder 2
    - currency_symbol: ''
      token_name: ''
  - 17000000


### 9. First bidder bids 30 Ada

In [48]:
AMOUNT_4=$((30 * ADA))

In [49]:
marlowe-cli run prepare --choice-party Role="Bidder 1"            \
                        --choice-name "Bid 1"                     \
                        --choice-number $AMOUNT_4                 \
                        --invalid-before $NOW                     \
                        --invalid-hereafter $LATER                \
                        --marlowe-file english-auction-08.marlowe \
                        --out-file     english-auction-09.marlowe \
                        --print-stats


Datum size: 921


In [50]:
jq .state.choices english-auction-09.marlowe | json2yaml

- - choice_name: Bid 3
    choice_owner:
      role_token: Bidder 3
  - 25000000
- - choice_name: Bid 2
    choice_owner:
      role_token: Bidder 2
  - 15000000
- - choice_name: Bid 1
    choice_owner:
      role_token: Bidder 1
  - 30000000


### 10. First bidder deposits 32 Ada

In [51]:
DELTA_4=$((AMOUNT_4 + MIN_ADA))
echo $DELTA_4

32000000


In [52]:
marlowe-cli run prepare --deposit-account Role="Bidder 1"         \
                        --deposit-party Role="Bidder 1"           \
                        --deposit-amount $DELTA_4                 \
                        --invalid-before $NOW                     \
                        --invalid-hereafter $LATER                \
                        --marlowe-file english-auction-09.marlowe \
                        --out-file     english-auction-10.marlowe \
                        --print-stats


Warnings:
  TransactionShadowing "Highest Bid" 25000000 30000000
Datum size: 877


In [53]:
jq .state.accounts english-auction-10.marlowe | json2yaml

- - - role_token: Seller
    - currency_symbol: ''
      token_name: ''
  - 2000000
- - - role_token: Seller
    - currency_symbol: 76574a57d6a0bc1db5ad534b90ac0aacdad765754eca3f4e7dc7c116
      token_name: Globe
  - 1
- - - role_token: Bidder 3
    - currency_symbol: ''
      token_name: ''
  - 27000000
- - - role_token: Bidder 2
    - currency_symbol: ''
      token_name: ''
  - 17000000
- - - role_token: Bidder 1
    - currency_symbol: ''
      token_name: ''
  - 32000000


### 11. Second bidder bids 40 Ada

In [54]:
AMOUNT_5=$((40 * ADA))

In [55]:
marlowe-cli run prepare --choice-party Role="Bidder 2"            \
                        --choice-name "Bid 2"                     \
                        --choice-number $AMOUNT_5                 \
                        --invalid-before $NOW                     \
                        --invalid-hereafter $LATER                \
                        --marlowe-file english-auction-10.marlowe \
                        --out-file     english-auction-11.marlowe \
                        --print-stats


Datum size: 949


In [56]:
jq .state.choices english-auction-11.marlowe | json2yaml

- - choice_name: Bid 3
    choice_owner:
      role_token: Bidder 3
  - 25000000
- - choice_name: Bid 2
    choice_owner:
      role_token: Bidder 2
  - 40000000
- - choice_name: Bid 1
    choice_owner:
      role_token: Bidder 1
  - 30000000


### 12. Second bidder deposits 25 Ada

The second bidder had previously deposited 15 Ada, so they only need to deposit 25 Ada more to match their bid of 40 Ada.

In [57]:
DELTA_5=$((AMOUNT_5 - AMOUNT_2))
echo $DELTA_5

25000000


In [58]:
marlowe-cli run prepare --deposit-account Role="Bidder 2"         \
                        --deposit-party Role="Bidder 2"           \
                        --deposit-amount $DELTA_5                 \
                        --invalid-before $NOW                     \
                        --invalid-hereafter $LATER                \
                        --marlowe-file english-auction-11.marlowe \
                        --out-file     english-auction-12.marlowe \
                        --print-stats


Warnings:
  TransactionShadowing "Highest Bid" 30000000 40000000
Datum size: 877


In [59]:
jq .state.accounts english-auction-12.marlowe | json2yaml

- - - role_token: Seller
    - currency_symbol: ''
      token_name: ''
  - 2000000
- - - role_token: Seller
    - currency_symbol: 76574a57d6a0bc1db5ad534b90ac0aacdad765754eca3f4e7dc7c116
      token_name: Globe
  - 1
- - - role_token: Bidder 3
    - currency_symbol: ''
      token_name: ''
  - 27000000
- - - role_token: Bidder 2
    - currency_symbol: ''
      token_name: ''
  - 42000000
- - - role_token: Bidder 1
    - currency_symbol: ''
      token_name: ''
  - 32000000


### 13. Third bidder bids 50 Ada

In [60]:
AMOUNT_6=$((50 * ADA))

In [61]:
marlowe-cli run prepare --choice-party Role="Bidder 3"            \
                        --choice-name "Bid 3"                     \
                        --choice-number $AMOUNT_6                 \
                        --invalid-before $NOW                     \
                        --invalid-hereafter $LATER                \
                        --marlowe-file english-auction-12.marlowe \
                        --out-file     english-auction-13.marlowe \
                        --print-stats


Datum size: 594


In [62]:
jq .state.choices english-auction-13.marlowe | json2yaml

- - choice_name: Bid 3
    choice_owner:
      role_token: Bidder 3
  - 50000000
- - choice_name: Bid 2
    choice_owner:
      role_token: Bidder 2
  - 40000000
- - choice_name: Bid 1
    choice_owner:
      role_token: Bidder 1
  - 30000000


### 14. Third bidder deposits 25 Ada, and contract closes

The third bidder needs to deposit an additional 25 Ada in order to fulfill their 50 Ada bid. Because this was the last round of bidding, the contract closes and makes payments to the seller and bidders.

In [63]:
DELTA_6=$((AMOUNT_6 - AMOUNT_3))
echo $DELTA_6

25000000


In [64]:
marlowe-cli run prepare --deposit-account Role="Bidder 3"         \
                        --deposit-party Role="Bidder 3"           \
                        --deposit-amount $DELTA_6                 \
                        --invalid-before $NOW                     \
                        --invalid-hereafter $LATER                \
                        --marlowe-file english-auction-13.marlowe \
                        --out-file     english-auction-14.marlowe \
                        --print-stats


Warnings:
  TransactionShadowing "Highest Bid" 40000000 50000000
Datum size: 128
Payment 1
  Acccount: "Bidder 3"
  Payee: Party "Seller"
  Ada: 50.000000
Payment 2
  Acccount: "Seller"
  Payee: Party "Bidder 3"
  Ada: 0.000000
  76574a57d6a0bc1db5ad534b90ac0aacdad765754eca3f4e7dc7c116."Globe": 1
Payment 3
  Acccount: "Seller"
  Payee: Party "Seller"
  Ada: 2.000000
Payment 4
  Acccount: "Bidder 3"
  Payee: Party "Bidder 3"
  Ada: 2.000000
Payment 5
  Acccount: "Bidder 2"
  Payee: Party "Bidder 2"
  Ada: 42.000000
Payment 6
  Acccount: "Bidder 1"
  Payee: Party "Bidder 1"
  Ada: 32.000000


We see that the first and second bidders receive their deposit back, the seller receives the 50 Ada of the winning bid, and the third bidder receives the native token. The accounts are now empty.

In [65]:
jq .state english-auction-14.marlowe | json2yaml

accounts: []
boundValues:
- - Highest Bid
  - 50000000
choices:
- - choice_name: Bid 3
    choice_owner:
      role_token: Bidder 3
  - 50000000
- - choice_name: Bid 2
    choice_owner:
      role_token: Bidder 2
  - 40000000
- - choice_name: Bid 1
    choice_owner:
      role_token: Bidder 1
  - 30000000
minTime: 1655691097000


The contract has closed.

In [66]:
jq .contract english-auction-14.marlowe | json2yaml

close
...


## Run the Contract on Chain

### Find the script addresses

In [67]:
MARLOWE_ADDRESS=$(jq -r .marloweValidator.address english-auction-01.marlowe)
echo $MARLOWE_ADDRESS

addr_test1wruj7yprp4dhw3qlmwg32hayxtg5e69f7fw8d68lhwxml0sy2a5nu


In [68]:
PAYOUT_ADDRESS=$(jq -r .rolesValidator.address english-auction-01.marlowe)
echo $PAYOUT_ADDRESS

addr_test1wrsfjxk2xsfk98w766a7872zntr7f0nnc3ujntuqq0ss8wsup09a2


### 1. Create the contract

In [69]:
TX_1=$(
marlowe-cli run execute --tx-in $(marlowe-cli util select --lovelace-only $((50 * ADA)) ${ADDRESS[0]} | sed -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/') \
                        --required-signer ${SKEY[0]} \
                        --change-address ${ADDRESS[0]} \
                        --marlowe-out-file english-auction-01.marlowe \
                        --out-file /dev/null \
                        --submit 600 \
                        --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TxId \"$TX_1\""


Fee: Lovelace 183849
Size: 404 / 32768 = 1%
Execution units:
  Memory: 0 / 30000000 = 0%
  Steps: 0 / 10000000000 = 0%
TxId "9a4616b9261614aa2dd88e11d72be5f519052981b53f11f383d975cab5a9337e"


In [70]:
echo "UTxOs at Marlowe validator address:"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$MARLOWE_ADDRESS"
echo
echo "UTxOs at Seller address:"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "${ADDRESS[0]}"

UTxOs at Marlowe validator address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
9a4616b9261614aa2dd88e11d72be5f519052981b53f11f383d975cab5a9337e     1        2000000 lovelace + TxOutDatumHash ScriptDataInAlonzoEra "f7b56ece8568464c604eb8e34b46a569a48fa1ea3ecda6e0837a2e8f8ad8efb0"

UTxOs at Seller address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
3aced0ba2210ae7a4e644d9285c986f08fea126b3cf353305bd85ee0db6e5440     4        10000000 lovelace + 1 d6390543e76fe398ccb5d7e90da5f0548e29759cf94d565d82ad4b19.53656c6c6572 + TxOutDatumNone
6db1c167b9f9d04dff91a1d5a5592c3cf210bf224d76b247f5726429d4ce7ca8     0        23811663 lovelace + TxOutDatumNone
9a4616b9261614aa2dd88e11d72be5f519052981b53f11f383d975cab5a9337e     0        57631818 lovela

### 2. Seller deposits the native asset

In [71]:
TX_2=$(
marlowe-cli run execute --marlowe-in-file english-auction-01.marlowe \
                        --tx-in-marlowe "$TX_1#1" \
                        --tx-in-collateral "$TX_1#0"\
                        --tx-in "$TX_1#0" \
                        --tx-in $(marlowe-cli util select --asset-only $TOKEN ${ADDRESS[0]} | sed -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/') \
                        --tx-in $(marlowe-cli util select --asset-only "${ROLE_TOKEN[0]}" ${ADDRESS[0]} | sed -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/') \
                        --required-signer ${SKEY[0]} \
                        --change-address ${ADDRESS[0]} \
                        --marlowe-out-file english-auction-02.marlowe \
                        --tx-out "${ADDRESS[0]}+$MIN_ADA+1 ${ROLE_TOKEN[0]}" \
                        --out-file /dev/null \
                        --submit 600 \
                        --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TxId \"$TX_2\""


Fee: Lovelace 1469233
Size: 14636 / 32768 = 44%
Execution units:
  Memory: 7791022 / 30000000 = 25%
  Steps: 2721412543 / 10000000000 = 27%
TxId "81205bcec1ca1903e24c8e8e7b0cd53f9312d61bd8bdcd8c829ddefec2145bb7"


In [72]:
echo "UTxOs at Marlowe validator address:"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$MARLOWE_ADDRESS"
echo
echo "UTxOs at Seller address:"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "${ADDRESS[0]}"

UTxOs at Marlowe validator address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
81205bcec1ca1903e24c8e8e7b0cd53f9312d61bd8bdcd8c829ddefec2145bb7     1        2000000 lovelace + 1 76574a57d6a0bc1db5ad534b90ac0aacdad765754eca3f4e7dc7c116.476c6f6265 + TxOutDatumHash ScriptDataInAlonzoEra "d6dc7ca600a94eb83802938458b0bfa05a8184df39bee9fb66e444c8c5ef9748"

UTxOs at Seller address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
6db1c167b9f9d04dff91a1d5a5592c3cf210bf224d76b247f5726429d4ce7ca8     0        23811663 lovelace + TxOutDatumNone
81205bcec1ca1903e24c8e8e7b0cd53f9312d61bd8bdcd8c829ddefec2145bb7     0        64662585 lovelace + TxOutDatumNone
81205bcec1ca1903e24c8e8e7b0cd53f9312d61bd8bdcd8c829ddefec2145bb7     2        2000000 lovelace 

### 3. Third bidder bids 5 Ada

In [73]:
TX_3=$(
marlowe-cli run execute --marlowe-in-file english-auction-02.marlowe \
                        --tx-in-marlowe "$TX_2#1" \
                        --tx-in-collateral $(marlowe-cli util select --lovelace-only $((10 * ADA)) ${ADDRESS[3]} | sed -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/') \
                        --tx-in $(marlowe-cli util select --lovelace-only $((10 * ADA)) ${ADDRESS[3]} | sed -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/') \
                        --tx-in $(marlowe-cli util select --asset-only "${ROLE_TOKEN[3]}" ${ADDRESS[3]} | sed -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/') \
                        --required-signer ${SKEY[3]} \
                        --change-address ${ADDRESS[3]} \
                        --marlowe-out-file english-auction-03.marlowe \
                        --tx-out "${ADDRESS[3]}+$MIN_ADA+1 ${ROLE_TOKEN[3]}" \
                        --out-file /dev/null \
                        --submit 600 \
                        --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TxId \"$TX_3\""


Fee: Lovelace 1652759
Size: 15526 / 32768 = 47%
Execution units:
  Memory: 9408720 / 30000000 = 31%
  Steps: 3490740496 / 10000000000 = 34%
TxId "98267509306a38274adf139e52d7b5a69b09154dce5e9a3a324b8077862448bf"


In [74]:
echo "UTxOs at Marlowe validator address:"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$MARLOWE_ADDRESS"
echo
echo "UTxOs at ${ROLE[3]} address:"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "${ADDRESS[3]}"

UTxOs at Marlowe validator address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
98267509306a38274adf139e52d7b5a69b09154dce5e9a3a324b8077862448bf     1        2000000 lovelace + 1 76574a57d6a0bc1db5ad534b90ac0aacdad765754eca3f4e7dc7c116.476c6f6265 + TxOutDatumHash ScriptDataInAlonzoEra "4281ced8013507068adb219e9d714250e78f0c0e50550d35be40e9a0bce62ac7"

UTxOs at Bidder 3 address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
98267509306a38274adf139e52d7b5a69b09154dce5e9a3a324b8077862448bf     0        96847241 lovelace + TxOutDatumNone
98267509306a38274adf139e52d7b5a69b09154dce5e9a3a324b8077862448bf     2        2000000 lovelace + 1 d6390543e76fe398ccb5d7e90da5f0548e29759cf94d565d82ad4b19.4269646465722033 + TxOutDatumNone
98267509306a3827

### 4. Third bidder deposits 5 Ada

In [75]:
TX_4=$(
marlowe-cli run execute --marlowe-in-file english-auction-03.marlowe \
                        --tx-in-marlowe "$TX_3#1" \
                        --tx-in-collateral "$TX_3#0" \
                        --tx-in "$TX_3#0" \
                        --tx-in "$TX_3#3" \
                        --tx-in "$TX_3#2" \
                        --required-signer ${SKEY[3]} \
                        --change-address ${ADDRESS[3]} \
                        --marlowe-out-file english-auction-04.marlowe \
                        --tx-out "${ADDRESS[3]}+$MIN_ADA+1 ${ROLE_TOKEN[3]}" \
                        --out-file /dev/null \
                        --submit 600 \
                        --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TxId \"$TX_4\""


Fee: Lovelace 1692487
Size: 15495 / 32768 = 47%
Execution units:
  Memory: 9937624 / 30000000 = 33%
  Steps: 3575768461 / 10000000000 = 35%
TxId "0317360099ef0b75398781af958c4365fb4ca2ccce4aa54b1e08f32240c3ef19"


In [76]:
echo "UTxOs at Marlowe validator address:"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$MARLOWE_ADDRESS"
echo
echo "UTxOs at ${ROLE[3]} address:"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "${ADDRESS[3]}"

UTxOs at Marlowe validator address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
0317360099ef0b75398781af958c4365fb4ca2ccce4aa54b1e08f32240c3ef19     1        9000000 lovelace + 1 76574a57d6a0bc1db5ad534b90ac0aacdad765754eca3f4e7dc7c116.476c6f6265 + TxOutDatumHash ScriptDataInAlonzoEra "11a2884cf76ec09dd6929a5ce8607afdf2e5af648aceace23ac7670dbb80aaf8"

UTxOs at Bidder 3 address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
0317360099ef0b75398781af958c4365fb4ca2ccce4aa54b1e08f32240c3ef19     0        88154754 lovelace + TxOutDatumNone
0317360099ef0b75398781af958c4365fb4ca2ccce4aa54b1e08f32240c3ef19     2        2000000 lovelace + 1 d6390543e76fe398ccb5d7e90da5f0548e29759cf94d565d82ad4b19.4269646465722033 + TxOutDatumNone
0317360099ef0b75

### 5. Second bidder bids 15 Ada

In [77]:
TX_5=$(
marlowe-cli run execute --marlowe-in-file english-auction-04.marlowe \
                        --tx-in-marlowe "$TX_4#1" \
                        --tx-in-collateral $(marlowe-cli util select --lovelace-only $((10 * ADA)) ${ADDRESS[2]} | sed -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/') \
                        --tx-in $(marlowe-cli util select --lovelace-only $((10 * ADA)) ${ADDRESS[2]} | sed -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/') \
                        --tx-in $(marlowe-cli util select --asset-only "${ROLE_TOKEN[2]}" ${ADDRESS[2]} | sed -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/') \
                        --required-signer ${SKEY[2]} \
                        --change-address ${ADDRESS[2]} \
                        --marlowe-out-file english-auction-05.marlowe \
                        --tx-out "${ADDRESS[2]}+$MIN_ADA+1 ${ROLE_TOKEN[2]}" \
                        --out-file /dev/null \
                        --submit 600 \
                        --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TxId \"$TX_5\""


Fee: Lovelace 1848768
Size: 16252 / 32768 = 49%
Execution units:
  Memory: 11257826 / 30000000 = 37%
  Steps: 4286461465 / 10000000000 = 42%
TxId "431ca4191e4f3bf8ade331743c4b3558786afde4f5558bb63fd76a596838b054"


In [78]:
echo "UTxOs at Marlowe validator address:"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$MARLOWE_ADDRESS"
echo
echo "UTxOs at ${ROLE[2]} address:"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "${ADDRESS[2]}"

UTxOs at Marlowe validator address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
431ca4191e4f3bf8ade331743c4b3558786afde4f5558bb63fd76a596838b054     1        9000000 lovelace + 1 76574a57d6a0bc1db5ad534b90ac0aacdad765754eca3f4e7dc7c116.476c6f6265 + TxOutDatumHash ScriptDataInAlonzoEra "3d29f2cc6abe998fa44bea7581f9ab6375f58853b81d655831926a4cb8ed8b79"

UTxOs at Bidder 2 address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
431ca4191e4f3bf8ade331743c4b3558786afde4f5558bb63fd76a596838b054     0        96651232 lovelace + TxOutDatumNone
431ca4191e4f3bf8ade331743c4b3558786afde4f5558bb63fd76a596838b054     2        2000000 lovelace + 1 d6390543e76fe398ccb5d7e90da5f0548e29759cf94d565d82ad4b19.4269646465722032 + TxOutDatumNone
431ca4191e4f3bf8

### 6. Second bidder deposits 15 Ada

In [79]:
TX_6=$(
marlowe-cli run execute --marlowe-in-file english-auction-05.marlowe \
                        --tx-in-marlowe "$TX_5#1" \
                        --tx-in-collateral "$TX_5#0" \
                        --tx-in "$TX_5#0" \
                        --tx-in "$TX_5#3" \
                        --tx-in "$TX_5#2" \
                        --required-signer ${SKEY[2]} \
                        --change-address ${ADDRESS[2]} \
                        --marlowe-out-file english-auction-06.marlowe \
                        --tx-out "${ADDRESS[2]}+$MIN_ADA+1 ${ROLE_TOKEN[2]}" \
                        --out-file /dev/null \
                        --submit 600 \
                        --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TxId \"$TX_6\""


Fee: Lovelace 1782938
Size: 15771 / 32768 = 48%
Execution units:
  Memory: 10880594 / 30000000 = 36%
  Steps: 3907210910 / 10000000000 = 39%
TxId "c24b821bc19a303990c65b8e2515b9ff759670792907744de2e2b4eb476dbaf0"


In [80]:
echo "UTxOs at Marlowe validator address:"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$MARLOWE_ADDRESS"
echo
echo "UTxOs at ${ROLE[2]} address:"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "${ADDRESS[2]}"

UTxOs at Marlowe validator address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
c24b821bc19a303990c65b8e2515b9ff759670792907744de2e2b4eb476dbaf0     1        26000000 lovelace + 1 76574a57d6a0bc1db5ad534b90ac0aacdad765754eca3f4e7dc7c116.476c6f6265 + TxOutDatumHash ScriptDataInAlonzoEra "69f2e1ab32b5652b65c6885e33b2aeada51d28be6ca6de3f5a48f96a93f20fd7"

UTxOs at Bidder 2 address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
c24b821bc19a303990c65b8e2515b9ff759670792907744de2e2b4eb476dbaf0     0        77868294 lovelace + TxOutDatumNone
c24b821bc19a303990c65b8e2515b9ff759670792907744de2e2b4eb476dbaf0     2        2000000 lovelace + 1 d6390543e76fe398ccb5d7e90da5f0548e29759cf94d565d82ad4b19.4269646465722032 + TxOutDatumNone
c24b821bc19a303

### 7. Third bidder bids 25 Ada

In [81]:
TX_7=$(
marlowe-cli run execute --marlowe-in-file english-auction-06.marlowe \
                        --tx-in-marlowe "$TX_6#1" \
                        --tx-in-collateral "$TX_4#0" \
                        --tx-in "$TX_4#0" \
                        --tx-in "$TX_4#3" \
                        --tx-in "$TX_4#2" \
                        --required-signer ${SKEY[3]} \
                        --change-address ${ADDRESS[3]} \
                        --marlowe-out-file english-auction-07.marlowe \
                        --tx-out "${ADDRESS[3]}+$MIN_ADA+1 ${ROLE_TOKEN[3]}" \
                        --out-file /dev/null \
                        --submit 600 \
                        --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TxId \"$TX_7\""


Fee: Lovelace 1961496
Size: 16376 / 32768 = 49%
Execution units:
  Memory: 12528882 / 30000000 = 41%
  Steps: 4695449193 / 10000000000 = 46%
TxId "8630155b4a57abc54c0206b62e4abed0e82ee70304e79c0345d611032d787d8e"


In [82]:
echo "UTxOs at Marlowe validator address:"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$MARLOWE_ADDRESS"
echo
echo "UTxOs at ${ROLE[3]} address:"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "${ADDRESS[3]}"

UTxOs at Marlowe validator address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
8630155b4a57abc54c0206b62e4abed0e82ee70304e79c0345d611032d787d8e     1        26000000 lovelace + 1 76574a57d6a0bc1db5ad534b90ac0aacdad765754eca3f4e7dc7c116.476c6f6265 + TxOutDatumHash ScriptDataInAlonzoEra "8714c69ae411f9e75c2c136504441cc3d7ab79f36bf38cbb48322ce8456fe5f5"

UTxOs at Bidder 3 address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
8630155b4a57abc54c0206b62e4abed0e82ee70304e79c0345d611032d787d8e     0        86193258 lovelace + TxOutDatumNone
8630155b4a57abc54c0206b62e4abed0e82ee70304e79c0345d611032d787d8e     2        2000000 lovelace + 1 d6390543e76fe398ccb5d7e90da5f0548e29759cf94d565d82ad4b19.4269646465722033 + TxOutDatumNone
8630155b4a57abc

### 8. Third bidder deposits 20 Ada

In [83]:
TX_8=$(
marlowe-cli run execute --marlowe-in-file english-auction-07.marlowe \
                        --tx-in-marlowe "$TX_7#1" \
                        --tx-in-collateral "$TX_7#0" \
                        --tx-in "$TX_7#0" \
                        --tx-in "$TX_7#3" \
                        --tx-in "$TX_7#2" \
                        --required-signer ${SKEY[3]} \
                        --change-address ${ADDRESS[3]} \
                        --marlowe-out-file english-auction-08.marlowe \
                        --tx-out "${ADDRESS[3]}+$MIN_ADA+1 ${ROLE_TOKEN[3]}" \
                        --out-file /dev/null \
                        --submit 600 \
                        --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TxId \"$TX_8\""


Fee: Lovelace 1817429
Size: 15799 / 32768 = 48%
Execution units:
  Memory: 11294092 / 30000000 = 37%
  Steps: 4037591190 / 10000000000 = 40%
TxId "3ea7fdba25d3bdd067ee9d256e69cec3fc8c3b06faacccf0953c4ece8b0d9334"


In [84]:
echo "UTxOs at Marlowe validator address:"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$MARLOWE_ADDRESS"
echo
echo "UTxOs at ${ROLE[3]} address:"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "${ADDRESS[3]}"

UTxOs at Marlowe validator address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
3ea7fdba25d3bdd067ee9d256e69cec3fc8c3b06faacccf0953c4ece8b0d9334     1        46000000 lovelace + 1 76574a57d6a0bc1db5ad534b90ac0aacdad765754eca3f4e7dc7c116.476c6f6265 + TxOutDatumHash ScriptDataInAlonzoEra "8df1637aba207e51d711784b74e1f9141e7d996c00ee2aec8ddaed4d76bee456"

UTxOs at Bidder 3 address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
3ea7fdba25d3bdd067ee9d256e69cec3fc8c3b06faacccf0953c4ece8b0d9334     0        64375829 lovelace + TxOutDatumNone
3ea7fdba25d3bdd067ee9d256e69cec3fc8c3b06faacccf0953c4ece8b0d9334     2        2000000 lovelace + 1 d6390543e76fe398ccb5d7e90da5f0548e29759cf94d565d82ad4b19.4269646465722033 + TxOutDatumNone
3ea7fdba25d3bdd

### 9. First bidder bids 30 Ada

In [85]:
TX_9=$(
marlowe-cli run execute --marlowe-in-file english-auction-08.marlowe \
                        --tx-in-marlowe "$TX_8#1" \
                        --tx-in-collateral $(marlowe-cli util select --lovelace-only $((10 * ADA)) ${ADDRESS[1]} | sed -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/') \
                        --tx-in $(marlowe-cli util select --lovelace-only $((10 * ADA)) ${ADDRESS[1]} | sed -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/') \
                        --tx-in $(marlowe-cli util select --asset-only "${ROLE_TOKEN[1]}" ${ADDRESS[1]} | sed -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/') \
                        --required-signer ${SKEY[1]} \
                        --change-address ${ADDRESS[1]} \
                        --marlowe-out-file english-auction-09.marlowe \
                        --tx-out "${ADDRESS[1]}+$MIN_ADA+1 ${ROLE_TOKEN[1]}" \
                        --out-file /dev/null \
                        --submit 600 \
                        --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TxId \"$TX_9\""


Fee: Lovelace 1897856
Size: 16368 / 32768 = 49%
Execution units:
  Memory: 11798586 / 30000000 = 39%
  Steps: 4463745661 / 10000000000 = 44%
TxId "7b634f7246089bbb63ab608c7689ee027d0f44b6d3f82216e57506edc7da4052"


In [86]:
echo "UTxOs at Marlowe validator address:"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$MARLOWE_ADDRESS"
echo
echo "UTxOs at ${ROLE[1]} address:"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "${ADDRESS[1]}"

UTxOs at Marlowe validator address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
7b634f7246089bbb63ab608c7689ee027d0f44b6d3f82216e57506edc7da4052     1        46000000 lovelace + 1 76574a57d6a0bc1db5ad534b90ac0aacdad765754eca3f4e7dc7c116.476c6f6265 + TxOutDatumHash ScriptDataInAlonzoEra "d389d87f09627bdb508ede61115da1a75b2792eec7eb9ae1e93829d731e2f69b"

UTxOs at Bidder 1 address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
7b634f7246089bbb63ab608c7689ee027d0f44b6d3f82216e57506edc7da4052     0        96602144 lovelace + TxOutDatumNone
7b634f7246089bbb63ab608c7689ee027d0f44b6d3f82216e57506edc7da4052     2        2000000 lovelace + 1 d6390543e76fe398ccb5d7e90da5f0548e29759cf94d565d82ad4b19.4269646465722031 + TxOutDatumNone
7b634f7246089bb

### 10. First bidder deposits 30 Ada

In [87]:
TX_10=$(
marlowe-cli run execute --marlowe-in-file english-auction-09.marlowe \
                        --tx-in-marlowe "$TX_9#1" \
                        --tx-in-collateral "$TX_9#0" \
                        --tx-in "$TX_9#0" \
                        --tx-in "$TX_9#3" \
                        --tx-in "$TX_9#2" \
                        --required-signer ${SKEY[1]} \
                        --change-address ${ADDRESS[1]} \
                        --marlowe-out-file english-auction-10.marlowe \
                        --tx-out "${ADDRESS[1]}+$MIN_ADA+1 ${ROLE_TOKEN[1]}" \
                        --out-file /dev/null \
                        --submit 600 \
                        --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TxId \"$TX_10\""


Fee: Lovelace 1830860
Size: 15883 / 32768 = 48%
Execution units:
  Memory: 11409160 / 30000000 = 38%
  Steps: 4080529510 / 10000000000 = 40%
TxId "6f3cfc1753ca162c8504a3650b951aa00a24000bd50797f753c9208d234ebb37"


In [88]:
echo "UTxOs at Marlowe validator address:"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$MARLOWE_ADDRESS"
echo
echo "UTxOs at ${ROLE[1]} address:"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "${ADDRESS[1]}"

UTxOs at Marlowe validator address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
6f3cfc1753ca162c8504a3650b951aa00a24000bd50797f753c9208d234ebb37     1        78000000 lovelace + 1 76574a57d6a0bc1db5ad534b90ac0aacdad765754eca3f4e7dc7c116.476c6f6265 + TxOutDatumHash ScriptDataInAlonzoEra "57faa637ef5ef5823b787ca1d7ce82a03a422a3618065954d7ea0e2ff5c66b3b"

UTxOs at Bidder 1 address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
6f3cfc1753ca162c8504a3650b951aa00a24000bd50797f753c9208d234ebb37     0        62771284 lovelace + TxOutDatumNone
6f3cfc1753ca162c8504a3650b951aa00a24000bd50797f753c9208d234ebb37     2        2000000 lovelace + 1 d6390543e76fe398ccb5d7e90da5f0548e29759cf94d565d82ad4b19.4269646465722031 + TxOutDatumNone
6f3cfc1753ca162

### 11. Second bidder bids 40 Ada

In [89]:
TX_11=$(
marlowe-cli run execute --marlowe-in-file english-auction-10.marlowe \
                        --tx-in-marlowe "$TX_10#1" \
                        --tx-in-collateral "$TX_6#0" \
                        --tx-in "$TX_6#0" \
                        --tx-in "$TX_6#3" \
                        --tx-in "$TX_6#2" \
                        --required-signer ${SKEY[2]} \
                        --change-address ${ADDRESS[2]} \
                        --marlowe-out-file english-auction-11.marlowe \
                        --tx-out "${ADDRESS[2]}+$MIN_ADA+1 ${ROLE_TOKEN[2]}" \
                        --out-file /dev/null \
                        --submit 600 \
                        --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TxId \"$TX_11\""


Fee: Lovelace 1994893
Size: 16488 / 32768 = 50%
Execution units:
  Memory: 12874236 / 30000000 = 42%
  Steps: 4813928844 / 10000000000 = 48%
TxId "cad8545c90f0f95dca4b5899026f2ec36b18e205b02960f11482b6e5ecbb86bd"


In [90]:
echo "UTxOs at Marlowe validator address:"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$MARLOWE_ADDRESS"
echo
echo "UTxOs at ${ROLE[2]} address:"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "${ADDRESS[2]}"

UTxOs at Marlowe validator address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
cad8545c90f0f95dca4b5899026f2ec36b18e205b02960f11482b6e5ecbb86bd     1        78000000 lovelace + 1 76574a57d6a0bc1db5ad534b90ac0aacdad765754eca3f4e7dc7c116.476c6f6265 + TxOutDatumHash ScriptDataInAlonzoEra "f7a346bf8bdd5df3e566597ab15481818a41c4a523e6b5504d664b0aff9b5cca"

UTxOs at Bidder 2 address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
cad8545c90f0f95dca4b5899026f2ec36b18e205b02960f11482b6e5ecbb86bd     0        75873401 lovelace + TxOutDatumNone
cad8545c90f0f95dca4b5899026f2ec36b18e205b02960f11482b6e5ecbb86bd     2        2000000 lovelace + 1 d6390543e76fe398ccb5d7e90da5f0548e29759cf94d565d82ad4b19.4269646465722032 + TxOutDatumNone
cad8545c90f0f95

### 12. Second bidder deposits 25 Ada

In [91]:
TX_12=$(
marlowe-cli run execute --marlowe-in-file english-auction-11.marlowe \
                        --tx-in-marlowe "$TX_11#1" \
                        --tx-in-collateral "$TX_11#0" \
                        --tx-in "$TX_11#0" \
                        --tx-in "$TX_11#3" \
                        --tx-in "$TX_11#2" \
                        --required-signer ${SKEY[2]} \
                        --change-address ${ADDRESS[2]} \
                        --marlowe-out-file english-auction-12.marlowe \
                        --tx-out "${ADDRESS[2]}+$MIN_ADA+1 ${ROLE_TOKEN[2]}" \
                        --out-file /dev/null \
                        --submit 600 \
                        --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TxId \"$TX_12\""


Fee: Lovelace 1865351
Size: 15911 / 32768 = 48%
Execution units:
  Memory: 11822658 / 30000000 = 39%
  Steps: 4210909790 / 10000000000 = 42%
TxId "461d7011023d8f2ed200f15d8c6c0386133122a796d4164a78a9c04e03e0d70e"


In [92]:
echo "UTxOs at Marlowe validator address:"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$MARLOWE_ADDRESS"
echo
echo "UTxOs at ${ROLE[2]} address:"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "${ADDRESS[2]}"

UTxOs at Marlowe validator address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
461d7011023d8f2ed200f15d8c6c0386133122a796d4164a78a9c04e03e0d70e     1        103000000 lovelace + 1 76574a57d6a0bc1db5ad534b90ac0aacdad765754eca3f4e7dc7c116.476c6f6265 + TxOutDatumHash ScriptDataInAlonzoEra "f8ab5fe2c9d00205930cd4eb63fbfabb054758ba2d0902546fff20736eac5f34"

UTxOs at Bidder 2 address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
461d7011023d8f2ed200f15d8c6c0386133122a796d4164a78a9c04e03e0d70e     0        49008050 lovelace + TxOutDatumNone
461d7011023d8f2ed200f15d8c6c0386133122a796d4164a78a9c04e03e0d70e     2        2000000 lovelace + 1 d6390543e76fe398ccb5d7e90da5f0548e29759cf94d565d82ad4b19.4269646465722032 + TxOutDatumNone
461d7011023d8f

### 13. Third bidder bids 50 Ada

In [93]:
TX_13=$(
marlowe-cli run execute --marlowe-in-file english-auction-12.marlowe \
                        --tx-in-marlowe "$TX_12#1" \
                        --tx-in-collateral "$TX_8#0" \
                        --tx-in "$TX_8#0" \
                        --tx-in "$TX_8#3" \
                        --tx-in "$TX_8#2" \
                        --required-signer ${SKEY[3]} \
                        --change-address ${ADDRESS[3]} \
                        --marlowe-out-file english-auction-13.marlowe \
                        --tx-out "${ADDRESS[3]}+$MIN_ADA+1 ${ROLE_TOKEN[3]}" \
                        --out-file /dev/null \
                        --submit 600 \
                        --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TxId \"$TX_13\""


Fee: Lovelace 1820148
Size: 15419 / 32768 = 47%
Execution units:
  Memory: 11490834 / 30000000 = 38%
  Steps: 4149759053 / 10000000000 = 41%
TxId "012d4ebba9e6378c988aaac3e799dbb154a63b79102318214a3cf2f0395b1bff"


In [94]:
echo "UTxOs at Marlowe validator address:"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$MARLOWE_ADDRESS"
echo
echo "UTxOs at ${ROLE[3]} address:"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "${ADDRESS[3]}"

UTxOs at Marlowe validator address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
012d4ebba9e6378c988aaac3e799dbb154a63b79102318214a3cf2f0395b1bff     1        103000000 lovelace + 1 76574a57d6a0bc1db5ad534b90ac0aacdad765754eca3f4e7dc7c116.476c6f6265 + TxOutDatumHash ScriptDataInAlonzoEra "0fd70fb37f3115ac16f67990f15e054443a0f79319c44aa514bf12df39515d44"

UTxOs at Bidder 3 address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
012d4ebba9e6378c988aaac3e799dbb154a63b79102318214a3cf2f0395b1bff     0        62555681 lovelace + TxOutDatumNone
012d4ebba9e6378c988aaac3e799dbb154a63b79102318214a3cf2f0395b1bff     2        2000000 lovelace + 1 d6390543e76fe398ccb5d7e90da5f0548e29759cf94d565d82ad4b19.4269646465722033 + TxOutDatumNone
012d4ebba9e637

### 14. Third bidder deposits 25 Ada, and contract closes

In [95]:
TX_14=$(
marlowe-cli run execute --marlowe-in-file english-auction-13.marlowe \
                        --tx-in-marlowe "$TX_13#1" \
                        --tx-in-collateral "$TX_13#0" \
                        --tx-in "$TX_13#0" \
                        --tx-in "$TX_13#3" \
                        --tx-in "$TX_13#2" \
                        --required-signer ${SKEY[3]} \
                        --change-address ${ADDRESS[3]} \
                        --marlowe-out-file english-auction-14.marlowe \
                        --tx-out "${ADDRESS[3]}+$MIN_ADA+1 ${ROLE_TOKEN[3]}" \
                        --out-file /dev/null \
                        --submit 600 \
                        --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TxId \"$TX_14\""


Fee: Lovelace 2050662
Size: 14490 / 32768 = 44%
Execution units:
  Memory: 15158376 / 30000000 = 50%
  Steps: 4978787464 / 10000000000 = 49%
TxId "68a733ab0c264e33c7c03cb1c04d9d538b348d1e4a00f285d7fb9ae9b843641e"


In [96]:
echo "UTxOs at Marlowe validator address:"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$MARLOWE_ADDRESS"
echo
echo "UTxOs at payout validator address:"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$PAYOUT_ADDRESS"
echo
echo "UTxOs at ${ROLE[3]} address:"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "${ADDRESS[3]}"

UTxOs at Marlowe validator address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------

UTxOs at payout validator address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
68a733ab0c264e33c7c03cb1c04d9d538b348d1e4a00f285d7fb9ae9b843641e     1        32000000 lovelace + TxOutDatumHash ScriptDataInAlonzoEra "b8f00d165b636528cb769f7830821eba1686f7ea947819fdb5dffc152f66d834"
68a733ab0c264e33c7c03cb1c04d9d538b348d1e4a00f285d7fb9ae9b843641e     2        42000000 lovelace + TxOutDatumHash ScriptDataInAlonzoEra "e9bbfbfb77c6833bf8d0c1019948c6c77d869129107611467ea4391373c9cb4f"
68a733ab0c264e33c7c03cb1c04d9d538b348d1e4a00f285d7fb9ae9b843641e     3        2000000 lovelace + 1 76574a57d6a0bc1db5ad534b90ac0aacdad765754eca3f4e7dc7c116.476c6f6265 + TxOutDatumH

### 15. Seller withdraws 52 Ada

The seller receives the 50 Ada winning bid plus the 2 Ada they originally deposited when the contract was created.

In [97]:
marlowe-cli run withdraw --role-name "${ROLE[0]}" \
                         --marlowe-file english-auction-14.marlowe \
                         --tx-in-collateral "$TX_2#0" \
                         --tx-in "$TX_2#0" \
                         --tx-in "$TX_2#3" \
                         --tx-in "$TX_2#2" \
                         --change-address ${ADDRESS[0]} \
                         --required-signer ${SKEY[0]} \
                         --tx-out "${ADDRESS[0]}+$MIN_ADA+1 ${ROLE_TOKEN[0]}" \
                         --out-file /dev/null \
                         --submit 600 \
                         --print-stats


Fee: Lovelace 455364
Size: 2929 / 32768 = 8%
Execution units:
  Memory: 1731660 / 30000000 = 5%
  Steps: 652951119 / 10000000000 = 6%
TxId "37a04bae0124dc97a4c8746e3fd6f9524a501c0527bcbd17292440c4df00f740"


In [98]:
echo "UTxOs at payout validator address:"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$PAYOUT_ADDRESS"
echo
echo "UTxOs at ${ROLE[0]} address:"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "${ADDRESS[0]}"

UTxOs at payout validator address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
68a733ab0c264e33c7c03cb1c04d9d538b348d1e4a00f285d7fb9ae9b843641e     1        32000000 lovelace + TxOutDatumHash ScriptDataInAlonzoEra "b8f00d165b636528cb769f7830821eba1686f7ea947819fdb5dffc152f66d834"
68a733ab0c264e33c7c03cb1c04d9d538b348d1e4a00f285d7fb9ae9b843641e     2        42000000 lovelace + TxOutDatumHash ScriptDataInAlonzoEra "e9bbfbfb77c6833bf8d0c1019948c6c77d869129107611467ea4391373c9cb4f"
68a733ab0c264e33c7c03cb1c04d9d538b348d1e4a00f285d7fb9ae9b843641e     3        2000000 lovelace + 1 76574a57d6a0bc1db5ad534b90ac0aacdad765754eca3f4e7dc7c116.476c6f6265 + TxOutDatumHash ScriptDataInAlonzoEra "4e3bf0fe2852adb2e4abee1d1b669106c769bd2b6e705fd848d6b591b888a57d"

UTxOs at Seller address:
                           TxHash                                 TxIx        Amount
-----

### 16. First bidder withdraws their 32 Ada deposit

In [99]:
marlowe-cli run withdraw --role-name "${ROLE[1]}" \
                         --marlowe-file english-auction-14.marlowe \
                         --tx-in-collateral "$TX_10#0" \
                         --tx-in "$TX_10#0" \
                         --tx-in "$TX_10#3" \
                         --tx-in "$TX_10#2" \
                         --change-address ${ADDRESS[1]} \
                         --required-signer ${SKEY[1]} \
                         --tx-out "${ADDRESS[1]}+$MIN_ADA+1 ${ROLE_TOKEN[1]}" \
                         --out-file /dev/null \
                         --submit 600 \
                         --print-stats


Fee: Lovelace 455540
Size: 2933 / 32768 = 8%
Execution units:
  Memory: 1731660 / 30000000 = 5%
  Steps: 652951119 / 10000000000 = 6%
TxId "155d3ef03f1f781fa8a9a0b33746cf871c4f01bdc3e77911c998ca0e87e679bd"


In [100]:
echo "UTxOs at payout validator address:"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$PAYOUT_ADDRESS"
echo
echo "UTxOs at ${ROLE[1]} address:"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "${ADDRESS[1]}"

UTxOs at payout validator address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
68a733ab0c264e33c7c03cb1c04d9d538b348d1e4a00f285d7fb9ae9b843641e     2        42000000 lovelace + TxOutDatumHash ScriptDataInAlonzoEra "e9bbfbfb77c6833bf8d0c1019948c6c77d869129107611467ea4391373c9cb4f"
68a733ab0c264e33c7c03cb1c04d9d538b348d1e4a00f285d7fb9ae9b843641e     3        2000000 lovelace + 1 76574a57d6a0bc1db5ad534b90ac0aacdad765754eca3f4e7dc7c116.476c6f6265 + TxOutDatumHash ScriptDataInAlonzoEra "4e3bf0fe2852adb2e4abee1d1b669106c769bd2b6e705fd848d6b591b888a57d"

UTxOs at Bidder 1 address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
155d3ef03f1f781fa8a9a0b33746cf871c4f01bdc3e77911c998ca0e87e679bd     0        63815744 lovelace + TxOutDatumNone
155d3

### 17. Second bidders withdraws their 42 Ada deposit

In [101]:
marlowe-cli run withdraw --role-name "${ROLE[2]}" \
                         --marlowe-file english-auction-14.marlowe \
                         --tx-in-collateral "$TX_12#0" \
                         --tx-in "$TX_12#0" \
                         --tx-in "$TX_12#3" \
                         --tx-in "$TX_12#2" \
                         --change-address ${ADDRESS[2]} \
                         --required-signer ${SKEY[2]} \
                         --tx-out "${ADDRESS[2]}+$MIN_ADA+1 ${ROLE_TOKEN[2]}" \
                         --out-file /dev/null \
                         --submit 600 \
                         --print-stats


Fee: Lovelace 449736
Size: 2933 / 32768 = 8%
Execution units:
  Memory: 1658656 / 30000000 = 5%
  Steps: 630869619 / 10000000000 = 6%
TxId "f1bd942bb27b3a694458fb1509708e9072b64690ebafe51b1a9bbfe1f4f34457"


In [102]:
echo "UTxOs at payout validator address:"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$PAYOUT_ADDRESS"
echo
echo "UTxOs at ${ROLE[2]} address:"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "${ADDRESS[2]}"

UTxOs at payout validator address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
68a733ab0c264e33c7c03cb1c04d9d538b348d1e4a00f285d7fb9ae9b843641e     3        2000000 lovelace + 1 76574a57d6a0bc1db5ad534b90ac0aacdad765754eca3f4e7dc7c116.476c6f6265 + TxOutDatumHash ScriptDataInAlonzoEra "4e3bf0fe2852adb2e4abee1d1b669106c769bd2b6e705fd848d6b591b888a57d"

UTxOs at Bidder 2 address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
f1bd942bb27b3a694458fb1509708e9072b64690ebafe51b1a9bbfe1f4f34457     0        50058314 lovelace + TxOutDatumNone
f1bd942bb27b3a694458fb1509708e9072b64690ebafe51b1a9bbfe1f4f34457     1        42000000 lovelace + TxOutDatumNone
f1bd942bb27b3a694458fb1509708e9072b64690ebafe51b1a9bbfe1f4f34457     2        2000000 lovelace

### 18. Third bidder withdraws the native token representing the asset, along with the minimum Ada

In [103]:
marlowe-cli run withdraw --role-name "${ROLE[3]}" \
                         --marlowe-file english-auction-14.marlowe \
                         --tx-in-collateral "$TX_14#0" \
                         --tx-in "$TX_14#0" \
                         --tx-in "$TX_14#5" \
                         --tx-in "$TX_14#6" \
                         --change-address ${ADDRESS[3]} \
                         --required-signer ${SKEY[3]} \
                         --tx-out "${ADDRESS[3]}+$MIN_ADA+1 ${ROLE_TOKEN[3]}" \
                         --out-file /dev/null \
                         --submit 600 \
                         --print-stats


Fee: Lovelace 474030
Size: 2973 / 32768 = 9%
Execution units:
  Memory: 1937564 / 30000000 = 6%
  Steps: 720211219 / 10000000000 = 7%
TxId "bf2fa15fc534eb850d0c193686b7c2243563a83b98a72767e4a35e412872bfda"


In [104]:
echo "UTxOs at payout validator address:"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$PAYOUT_ADDRESS"
echo
echo "UTxOs at ${ROLE[3]} address:"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "${ADDRESS[3]}"

UTxOs at payout validator address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------

UTxOs at Bidder 3 address:
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
bf2fa15fc534eb850d0c193686b7c2243563a83b98a72767e4a35e412872bfda     0        36530989 lovelace + TxOutDatumNone
bf2fa15fc534eb850d0c193686b7c2243563a83b98a72767e4a35e412872bfda     1        2000000 lovelace + 1 76574a57d6a0bc1db5ad534b90ac0aacdad765754eca3f4e7dc7c116.476c6f6265 + TxOutDatumNone
bf2fa15fc534eb850d0c193686b7c2243563a83b98a72767e4a35e412872bfda     2        2000000 lovelace + 1 d6390543e76fe398ccb5d7e90da5f0548e29759cf94d565d82ad4b19.4269646465722033 + TxOutDatumNone
